In [12]:
import os
import nrrd
import numpy as np
import h5py
from PIL import Image, ImageFilter
import scipy.ndimage
import SimpleITK as sitk
import urllib, urllib2
import json
import pandas as pd
import zipfile
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from gene_surfaceview import *
from mayavi import mlab


In [4]:
rma_url = ("http://api.brain-map.org/api/v2/data/query.json?criteria=" +
           "model::SectionDataSet,rma::criteria,[failed$eqFalse]," +
           "plane_of_section[name$eqcoronal],products[name$eq'Mouse%20Brain']," +
           "treatments[name$eqISH],rma::include,genes," +
           "rma::options[only$eq'genes.acronym,id'][num_rows$eqall]")

response = urllib2.urlopen(rma_url).read()
data = json.loads(response)
sections = {v["id"]: v["genes"][0]["acronym"] for v in data["msg"]}


In [5]:
for section_id in sections:
    print section_id
    gene_array = get_section_image(section_id)
    
    # for visualization
    gene_array[np.where(gene_array==-1)] = 0
    mlab.pipeline.volume(mlab.pipeline.scalar_field(gene_array))
    mlab.show()    
    break

73994713


# size of the image

In [25]:
gene_array.shape

(67, 41, 58)

# number of genes

In [26]:
len(sections)

4345

# how many genes are in the RNA_Seq data set?

In [27]:
genes = pd.read_csv('../data/mouse_VISp_2018-06-14_genes-rows.csv') # gene information
genes.head()

,gene_symbol,gene_id,chromosome,gene_entrez_id,gene_name
0,0610005C13Rik,500717483,7,71661,RIKEN cDNA 0610005C13 gene
1,0610006L08Rik,500717917,7,76253,RIKEN cDNA 0610006L08 gene
2,0610007P14Rik,500730104,12,58520,RIKEN cDNA 0610007P14 gene
3,0610009B22Rik,500726890,11,66050,RIKEN cDNA 0610009B22 gene
4,0610009E02Rik,500702775,2,100125929,RIKEN cDNA 0610009E02 gene


In [28]:
symbols = np.unique(genes['gene_symbol'])

In [29]:
len(set(sections.values()).intersection(set(symbols))) * 1. / len(set(sections.values()))

0.9750122488975992

***should do more to explore the relationship between these two sets***